# 1. Amazon SageMaker 모델 학습 실습

## 1. 모델 훈련 개요

이 노트북은 SageMaker에서 Training Job을 통해서 모델 학습을 합니다.상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 학습](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script)

- 일반적으로 크게 3단계로 진행이 됩니다.
    - (1) 학습 실행 작업 정의
        - 학습 코드
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - (2) 학습 데이터셋 지정
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - (3) 학습 실행

### 사용 라이브러리 import

In [67]:
import boto3
import sagemaker

from sagemaker.xgboost.estimator import XGBoost

### SageMaker 세션과 Role, 사용할 버킷 정의

In [68]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

In [69]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/code'
output_path = f's3://{bucket}/output'

### 하이퍼파라미터 정의

In [80]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### (1) 학습 실행 작업 정의

In [81]:
xgb_estimator = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_code_path,
    hyperparameters = hyperparameters,
    role = role,
    sagemaker_session = sagemaker_session,
    instance_count = 1,
    instance_type = "ml.m5.xlarge",
    framework_version = "1.3-1")

### (2) 학습 데이터셋 지정

In [82]:
data_path=f's3://{bucket}/dataset'
!aws s3 sync ./dataset/ $data_path

### (3) 학습 실행

In [83]:
xgb_estimator.fit(inputs = {'train': data_path + "/train/train.csv"},
                  wait=False)

In [84]:
xgb_estimator.logs()

2022-03-08 03:01:40 Starting - Starting the training job...
2022-03-08 03:02:04 Starting - Preparing the instances for trainingProfilerReport-1646708500: InProgress
......
2022-03-08 03:03:06 Downloading - Downloading input data...
2022-03-08 03:03:32 Training - Downloading the training image.....[2022-03-08 03:04:16.027 ip-10-0-171-105.us-west-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-03-08:03:04:16:INFO] Imported framework sagemaker_xgboost_container.training
[2022-03-08:03:04:16:INFO] No GPUs detected (normal if no gpus installed)
[2022-03-08:03:04:16:INFO] Invoking user training script.
[2022-03-08:03:04:16:INFO] Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
[2022-03-08:03:04:16:INFO] Generating setup.cfg
[2022-03-08:03:04:16:INFO] Generating MANIFEST.in
[2022-03-08:03:04:16:INFO] Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Preparing metadat